In [4]:
# https://chat.openai.com/share/2894d320-6996-4867-8596-afd12be26236
import workout_log as wl
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split


valid_movements = [
    "rest", # 0 for padding
    "alternating pistols",
    "burpees",
    "chest to bars",
    "deadlifts",
    "double unders",
    "dual kb front rack step-ups, 20\" box",
    "dumbbell walking lunges",
    "dumbbell snatch",
    "front rack lunges",
    "front squats",
    "handstand pushups",
    "hang power cleans",
    "jumping air squats",
    "kettlebell swings",
    "power cleans",
    "pullups",
    "push jerks",
    "pushups",
    "ring dips",
    "running",
    "strict pullups",
]



In [7]:
# Combine all movements for each date into a single textual feature
maxLen = 0
for workout in wl.workout_log:
    maxLen = max(maxLen, len(workout["movements"]))

X = np.empty((len(wl.workout_log), maxLen*3 + 1))
Y = np.empty(len(wl.workout_log))

for workoutIdx, workout in enumerate(wl.workout_log):
    X[workoutIdx, 0] = workout["date"].toordinal()
    Y[workoutIdx] = workout["time_minutes"]
    for movementIdx, movement in enumerate(workout["movements"]):
        X[workoutIdx, movementIdx*3 + 1] = float(valid_movements.index(movement["movement"]))
        X[workoutIdx, movementIdx*3 + 2] = float(movement["repetitions"])
        X[workoutIdx, movementIdx*3 + 3] = float(movement["weight"])

In [8]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [9]:
# Create a TensorFlow model
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(len(X[0]),)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1))  # Predicting a single numerical value

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=4)

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 441ms/step - loss: 491606496.0000 - mae: 20944.4141 - val_loss: 10701138.0000 - val_mae: 3271.2324
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 62478612.0000 - mae: 6492.4355 - val_loss: 262425056.0000 - val_mae: 16199.5342
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 249704256.0000 - mae: 15788.5205 - val_loss: 112826736.0000 - val_mae: 10621.9863
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 83977720.0000 - mae: 8779.7158 - val_loss: 757057.6250 - val_mae: 869.9905
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 11956170.0000 - mae: 2510.9912 - val_loss: 86159760.0000 - val_mae: 9282.2178
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 93927416.0000 - mae: 9671.7910 - val_loss: 97884216.0000 - val_mae: 9893.6357
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 84217416.0000 - mae: 9111.6855 - val_loss: 19169436.0000 - val_mae: 4378.2720
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step

In [8]:
# Make predictions on new data
#new_workouts = X_test[0:2]
new_workouts = X_train[0:2]

# Predict and output results
predictions = model.predict(new_workouts)
print("Predictions:", predictions)
print("Actual:", y_train[0:2])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
Predictions: [[-14083.821 ]
 [-14081.9795]]
Actual: [12.85       10.53333333]
